In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Cargar los datos (Codificación del CSV latin1)

In [5]:
df = pd.read_csv(r'C:\Users\albsa\Downloads\total_delitos_municipio.csv', encoding='latin1')

Imprimir las primeras 5 filas para observar el formato del CSV

In [6]:
print(df.head())

          Entidad  Cve. Municipio       Municipio Tipo de delito  Total
0  Aguascalientes            1001  Aguascalientes           Robo   1436
1  Aguascalientes            1001  Aguascalientes      Secuestro      0
2  Aguascalientes            1002        Asientos           Robo     24
3  Aguascalientes            1002        Asientos      Secuestro      0
4  Aguascalientes            1003        Calvillo           Robo     33


Kmeans con 3 clases (Bajo, Medio, Alto) considerando los valores 0,0 en delitos y robos

In [7]:

# 1. Preparar los datos para clustering
# Crear un DataFrame pivote con Robos y Secuestros por municipio
df_pivot = df.pivot_table(
    index=['Entidad', 'Cve. Municipio', 'Municipio'],
    columns='Tipo de delito',
    values='Total',
    fill_value=0
).reset_index()

# Renombrar columnas para claridad
df_pivot.columns.name = None
df_pivot = df_pivot.rename(columns={'Robo': 'Total_Robos', 'Secuestro': 'Total_Secuestros'})

# 2. Crear matriz de características para clustering
X = df_pivot[['Total_Robos', 'Total_Secuestros']].values

# 3. Escalar los datos (importante para K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Aplicar K-Means con 3 clusters
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# 5. Asignar etiquetas a los clusters
# Ordenar clusters por nivel de delitos (de menor a mayor)
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
total_centers = cluster_centers.sum(axis=1)
sorted_indices = np.argsort(total_centers)

# Mapear clusters a etiquetas
cluster_labels = {
    sorted_indices[0]: 'Bajo',
    sorted_indices[1]: 'Medio',
    sorted_indices[2]: 'Alto'
}

df_pivot['Nivel_Delitos'] = [cluster_labels[c] for c in clusters]

# 6. Fusionar con el DataFrame original
df_final = df.merge(
    df_pivot[['Entidad', 'Cve. Municipio', 'Nivel_Delitos']],
    on=['Entidad', 'Cve. Municipio'],
    how='left'
)

# 7. Verificar resultados
print("\nDistribución de niveles de delitos:")
print(df_final['Nivel_Delitos'].value_counts())

print("\nCentroides de los clusters (valores originales):")
print(pd.DataFrame(cluster_centers, columns=['Total_Robos', 'Total_Secuestros']))

print("\nMuestra del DataFrame final con niveles de riesgo:")
print(df_final.head())


Distribución de niveles de delitos:
Nivel_Delitos
Bajo     4884
Alto       86
Medio       2
Name: count, dtype: int64

Centroides de los clusters (valores originales):
   Total_Robos  Total_Secuestros
0    19.532351          0.018837
1  1203.418605          0.860465
2  1005.000000         17.000000

Muestra del DataFrame final con niveles de riesgo:
          Entidad  Cve. Municipio       Municipio Tipo de delito  Total  \
0  Aguascalientes            1001  Aguascalientes           Robo   1436   
1  Aguascalientes            1001  Aguascalientes      Secuestro      0   
2  Aguascalientes            1002        Asientos           Robo     24   
3  Aguascalientes            1002        Asientos      Secuestro      0   
4  Aguascalientes            1003        Calvillo           Robo     33   

  Nivel_Delitos  
0          Alto  
1          Alto  
2          Bajo  
3          Bajo  
4          Bajo  


Kmeans con 3 clases (Bajo, Medio, Alto); sin considerar los valores 0,0 en delitos y robos y asignandoles la eqtiueta "Bajo"

In [8]:
# 1. Preparar los datos para clustering
df_pivot = df.pivot_table(
    index=['Entidad', 'Cve. Municipio', 'Municipio'],
    columns='Tipo de delito',
    values='Total',
    fill_value=0
).reset_index()

df_pivot.columns.name = None
df_pivot = df_pivot.rename(columns={'Robo': 'Total_Robos', 'Secuestro': 'Total_Secuestros'})

# 2. Identificar municipios con ambos delitos = 0 (para etiqueta "Bajo")
condicion_bajo = (df_pivot['Total_Robos'] == 0) & (df_pivot['Total_Secuestros'] == 0)
df_bajo = df_pivot[condicion_bajo].copy()
df_bajo['Nivel_Delitos'] = 'Bajo'

# 3. Filtrar municipios con al menos un delito > 0
df_activos = df_pivot[~condicion_bajo].copy()

# 4. Aplicar clustering solo a municipios con delitos
if not df_activos.empty:
    X = df_activos[['Total_Robos', 'Total_Secuestros']].values
    
    # Escalado robusto (considerando la alta presencia de ceros)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Determinar clusters (mínimo 2, máximo 3)
    n_clusters = min(3, max(2, len(df_activos) // 10))  # Asegura clusters significativos
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    clusters = kmeans.fit_predict(X_scaled)
    
    # Asignar etiquetas de riesgo
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
    total_centers = cluster_centers.sum(axis=1)
    sorted_indices = np.argsort(total_centers)
    
    # Mapeo dinámico de etiquetas
    risk_labels = ['Medio', 'Alto'] if n_clusters == 2 else ['Bajo', 'Medio', 'Alto']
    cluster_labels = {sorted_indices[i]: risk_labels[i] for i in range(n_clusters)}
    
    df_activos['Nivel_Delitos'] = [cluster_labels[c] for c in clusters]
else:
    df_activos['Nivel_Delitos'] = 'Bajo'  # Si todos son cero

# 5. Combinar resultados
df_clusters = pd.concat([df_activos, df_bajo], ignore_index=True)

# 6. Fusionar con el DataFrame original
df_final = df.merge(
    df_clusters[['Entidad', 'Cve. Municipio', 'Nivel_Delitos']],
    on=['Entidad', 'Cve. Municipio'],
    how='left'
)

# 7. Resultados
print("\nDistribución de niveles de delitos:")
print(df_final['Nivel_Delitos'].value_counts())

if not df_activos.empty and 'cluster_centers' in locals():
    print("\nCentroides de los clusters (valores originales):")
    print(pd.DataFrame(cluster_centers, columns=['Total_Robos', 'Total_Secuestros']))

print("\nMuestra del DataFrame final:")
print(df_final.head())


Distribución de niveles de delitos:
Nivel_Delitos
Bajo     4894
Alto       76
Medio       2
Name: count, dtype: int64

Centroides de los clusters (valores originales):
   Total_Robos  Total_Secuestros
0    31.993598          0.035851
1  1301.868421          0.710526
2  1005.000000         17.000000

Muestra del DataFrame final:
          Entidad  Cve. Municipio       Municipio Tipo de delito  Total  \
0  Aguascalientes            1001  Aguascalientes           Robo   1436   
1  Aguascalientes            1001  Aguascalientes      Secuestro      0   
2  Aguascalientes            1002        Asientos           Robo     24   
3  Aguascalientes            1002        Asientos      Secuestro      0   
4  Aguascalientes            1003        Calvillo           Robo     33   

  Nivel_Delitos  
0          Alto  
1          Alto  
2          Bajo  
3          Bajo  
4          Bajo  


Kmeans con 4 clases (Muy Bajo, Bajo, Medio, Alto); sin considerar los valores 0,0 en delitos y robos y asignandoles la eqtiueta "Muy Bajo"

In [9]:
# 1. Preparar los datos para clustering
df_pivot = df.pivot_table(
    index=['Entidad', 'Cve. Municipio', 'Municipio'],
    columns='Tipo de delito',
    values='Total',
    fill_value=0
).reset_index()

df_pivot.columns.name = None
df_pivot = df_pivot.rename(columns={'Robo': 'Total_Robos', 'Secuestro': 'Total_Secuestros'})

# 2. Identificar municipios con ambos delitos = 0 (para etiqueta "Muy Bajo")
condicion_muy_bajo = (df_pivot['Total_Robos'] == 0) & (df_pivot['Total_Secuestros'] == 0)
df_muy_bajo = df_pivot[condicion_muy_bajo].copy()
df_muy_bajo['Nivel_Delitos'] = 'Muy Bajo'

# 3. Filtrar municipios con al menos un delito > 0
df_activos = df_pivot[~condicion_muy_bajo].copy()

# 4. Aplicar clustering solo a municipios con delitos
if not df_activos.empty:
    X = df_activos[['Total_Robos', 'Total_Secuestros']].values
    
    # Escalado de datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Determinar clusters dinámicamente
    n_clusters = min(3, max(2, len(df_activos) // 10))
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
    clusters = kmeans.fit_predict(X_scaled)
    
    # Asignar etiquetas de riesgo jerárquicas
    cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
    total_centers = cluster_centers.sum(axis=1)
    sorted_indices = np.argsort(total_centers)
    
    # Mapeo dinámico de etiquetas (siempre comenzando desde "Bajo")
    risk_labels = ['Bajo', 'Medio', 'Alto'][:n_clusters]
    cluster_labels = {sorted_indices[i]: risk_labels[i] for i in range(n_clusters)}
    
    df_activos['Nivel_Delitos'] = [cluster_labels[c] for c in clusters]
else:
    df_activos['Nivel_Delitos'] = 'Bajo'  # Caso extremo donde todos son 0

# 5. Combinar resultados
df_clusters = pd.concat([df_activos, df_muy_bajo], ignore_index=True)

# 6. Fusionar con el DataFrame original
df_final = df.merge(
    df_clusters[['Entidad', 'Cve. Municipio', 'Nivel_Delitos']],
    on=['Entidad', 'Cve. Municipio'],
    how='left'
)

# 7. Ordenar niveles para mejor presentación
nivel_orden = ['Muy Bajo', 'Bajo', 'Medio', 'Alto']
df_final['Nivel_Delitos'] = pd.Categorical(
    df_final['Nivel_Delitos'],
    categories=nivel_orden,
    ordered=True
)

# 8. Resultados y análisis
print("\nDistribución de niveles de delitos:")
print(df_final['Nivel_Delitos'].value_counts().sort_index())

if not df_activos.empty and 'cluster_centers' in locals():
    print("\nCentroides de los clusters (valores originales):")
    centroides_df = pd.DataFrame(cluster_centers, columns=['Total_Robos', 'Total_Secuestros'])
    centroides_df['Nivel_Asignado'] = risk_labels
    print(centroides_df.sort_values(by=['Total_Robos', 'Total_Secuestros']))

print("\nMuestra del DataFrame final:")
print(df_final.sort_values(['Entidad', 'Cve. Municipio']).head(10))


Distribución de niveles de delitos:
Nivel_Delitos
Muy Bajo    1770
Bajo        3124
Medio          2
Alto          76
Name: count, dtype: int64

Centroides de los clusters (valores originales):
   Total_Robos  Total_Secuestros Nivel_Asignado
0    31.993598          0.035851           Bajo
2  1005.000000         17.000000           Alto
1  1301.868421          0.710526          Medio

Muestra del DataFrame final:
           Entidad  Cve. Municipio       Municipio Tipo de delito  Total  \
0   Aguascalientes            1001  Aguascalientes           Robo   1436   
1   Aguascalientes            1001  Aguascalientes      Secuestro      0   
2   Aguascalientes            1002        Asientos           Robo     24   
3   Aguascalientes            1002        Asientos      Secuestro      0   
4   Aguascalientes            1003        Calvillo           Robo     33   
5   Aguascalientes            1003        Calvillo      Secuestro      0   
6   Aguascalientes            1004          CosÃ­o 